In [8]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import time
from splinter import Browser
import pandas as pd

In [9]:
# @NOTE: Replace the path with your actual path to the chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

**NASA Mars News**

In [10]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(3)

In [11]:
# Create BeautifulSoup object; parse with 'lxml'
html = browser.html
soup = bs(html, "lxml")

In [12]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
#Assign the text to variables that you can reference later.
news_title=soup.find('div', class_='content_title').a.text
news_p = soup.find('div', class_='article_teaser_body').text

print(f"News Title:{news_title}")
print()
print(f"Paragraph Text:{news_p}")

News Title:After a Reset, Curiosity Is Operating Normally

Paragraph Text:Curiosity has returned to science operations and is once again exploring the clay unit. 


**JPL Mars Space Images - Featured Image**

In [13]:
#Use splinter to navigate the site and find the image url for the current Featured
#Mars Image and assign the url string to a variable called featured_image_url.

#browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(5)

html = browser.html
soup = bs(html, "lxml")
img_src=soup.find("ul",class_="articles").li.find("div",class_="img").img['src']
featured_image_url=f"https://www.jpl.nasa.gov{img_src}"
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23054-640x350.jpg


**Mars Weather**

In [14]:
#Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet
#from the page. Save the tweet text for the weather report as a variable called mars_weather.
url="https://twitter.com/marswxreport?lang=en"
response = requests.get(url)
soup = bs(response.text, 'lxml')
mars_weather =soup.find("div",class_="js-tweet-text-container").p.text.rsplit(' ', 1)[0]
print(mars_weather)

InSight sol 92 (2019-03-01) low -94.4ºC (-137.9ºF) high -12.9ºC (8.8ºF)
winds from the SW at 4.6 m/s (10.2 mph) gusting to 10.4 m/s (23.2 mph)
pressure at 7.20


**Mars Facts**

In [16]:
#Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
#Use Pandas to convert the data to a HTML table string.
url="https://space-facts.com/mars/"
tables = pd.read_html(url)
df_mars = tables[0]
df_mars.columns = ['Description', 'Value']
df_mars.set_index('Description',inplace=True)
df_mars.to_html('mars.html')
df_mars

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


**Mars Hemispheres**

In [17]:
#Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
#You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
hemisphere_image_urls =[]

#browser = Browser('chrome', **executable_path, headless=False)
url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)
html = browser.html
soup = bs(html, "lxml")
img_divs=soup.find("div",class_="collapsible results").find_all('div',class_="item")
for item in img_divs:
    title=item.h3.text
    browser.click_link_by_partial_text(title)
    time.sleep(1)
    html = browser.html
    soup = bs(html, 'lxml')
    image_url_full=soup.find("div",class_="downloads").a['href']
    #print(title)
    #print(image_url_full)
    hemisphere_image_urls.append({'title':title, 'img_url':image_url_full })
    browser.click_link_by_partial_text("Back")
    time.sleep(1)
browser.quit()

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]